In [1]:
import os

import random
import math

import tensorflow as tf

from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from PIL import Image

import keras.backend as K

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [3]:
def Conv2D_Input(filters, kernel_size, strides, padding, activation, kernel_regularizer, name, input_shape):
    return tf.keras.layers.Conv2D(filters = filters, kernel_size = kernel_size, 
                                  strides = strides, padding = padding, activation = activation, 
                                  kernel_regularizer = kernel_regularizer, name = name, 
                                  input_shape = input_shape)


def Conv2D(filters, kernel_size, strides, padding, activation, kernel_regularizer, name):
    return tf.keras.layers.Conv2D(filters = filters, kernel_size = kernel_size, 
                                  strides = strides, padding = padding, activation = activation, 
                                  kernel_regularizer = kernel_regularizer, name = name)

def MaxPool(pool_size, strides, padding, name):
    return tf.keras.layers.MaxPooling2D(pool_size = pool_size, strides = strides, 
                                        padding = padding, name = name)

def Flatten(name):
    return tf.keras.layers.Flatten(name = name)

def Dense(units, activation, kernel_regularizer, name):
    return tf.keras.layers.Dense(units = units, activation = activation, 
                                 kernel_regularizer=kernel_regularizer,name = name)

def Batch_Normalize():
    return tf.keras.layers.BatchNormalization(axis= -1)

def DropOut(dropout_rate):
    return tf.keras.layers.Dropout(dropout_rate)

In [4]:
def prepare_callbacks(es_patience):
    callback_early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=es_patience,
                                                               verbose=1, restore_best_weights=True)
    callback_model= ModelCheckpoint(filepath='/home/ubuntu/kaggle/flowers/xray_images/Outputs/Best_Model', 
                                    save_best_only=True, save_weights_only=True)
    callbacks = [callback_early_stopping] #, callback_model]
    return callbacks

In [5]:
def compile_and_run_model(Model, Opt_Algo, LR, Mtm, Loss, Metrics):
    
    if Opt_Algo == 'sgd':
        Optimizer = tf.keras.optimizers.SGD(LR, Mtm)
        Model.compile(optimizer = Optimizer, loss = Loss, metrics = Metrics)
    
    elif Opt_Algo == 'adam':
        Optimizer = tf.keras.optimizers.Adam(learning_rate = LR)
        Model.compile(optimizer = Optimizer, loss = Loss, metrics = Metrics)

In [6]:
def prepare_train_test_data(train_dir, test_dir, batch_size, input_size):
    
    train_datagen = ImageDataGenerator(rescale=1.0/255.)
    test_datagen = ImageDataGenerator(rescale=1.0/255.)
    
    train_gen = train_datagen.flow_from_directory(train_dir,batch_size=batch_size,color_mode= 'rgb',
                                                    class_mode='binary',target_size=input_size) 
    test_gen = test_datagen.flow_from_directory(test_dir,batch_size=batch_size,color_mode= 'rgb',
                                                    class_mode='binary',target_size=input_size)
    
    return train_gen, test_gen

In [7]:
def export_results(history, hyperparams, File_Name1, File_Name2):
    
    file = open(File_Name1,"w")
    file.write(str(hyperparams))
    file.close()
    
    df = pd.DataFrame.from_dict(history.history.items())
    
    df0 = pd.DataFrame()
    for i in range(7):
        df1 = pd.DataFrame(df[1][i],columns = [df[0][i]])
        df0 = pd.concat([df0, df1], axis=1)
    df0.to_csv(File_Name2)
    
    print("Model parameters and results have been exported.")

In [8]:
def run_model(param_values, train_dir, test_dir, early_stopping, epochs):
    
    hyperparams = set_hyperparameters(param_values)
    Model = generate_model(hyperparams)
    
    Model.summary()

    print("")
    print("Batch_Size: "+str(hyperparams['BS']))
    print("Learning_Rate: "+str(hyperparams['LR']))
    print("Momentum: "+str(hyperparams['Mtm']))
    print("Optimization_Algo: "+str(hyperparams['Opt_Algo']))
    print("Regularization_Cv_Layers" + str(hyperparams['Cv_Reg']))
    print("Regularization_Dense_Layers" + str(hyperparams['Dl_Reg']))
    print ()
    print("")

    train_gen, test_gen = prepare_train_test_data(train_dir, test_dir, 
                                                  hyperparams['BS'], hyperparams['Input_Shape_WO_Channel'])
    
    compile_and_run_model(Model, hyperparams['Opt_Algo'], hyperparams['LR'], 
                          hyperparams['Mtm'], hyperparams['Loss'], 
                          hyperparams['Metrics'])
    
    callbacks = prepare_callbacks(hyperparams['Early_Stopping'])
    
    history = Model.fit(train_gen, validation_data= test_gen,
                        epochs = hyperparams['Epochs'], verbose=1, callbacks = callbacks)
    
    File_Name1 = 'Hyperparams_Model_Evaluation.txt'
    File_Name2 = 'Model_Results_Evaluation.csv'
    
    export_results(history, hyperparams, File_Name1, File_Name2)
    return Model, history, hyperparams

In [9]:
def set_hyperparameters():
    
    LR = pow(10,-3.9)
    Mtm = 0
    BS = 64
    
    Cv_Bks = 3
    Cv_CvLy = [2, 2, 1]
    Cv_PoLy = [1, 1, 1]

    Cv_Fil = [32, 64, 96]
    Cv_KS =  add_for_each_block(3,Cv_Bks)
    Cv_Reg = add_for_each_block(None,Cv_Bks)
    Cv_Pd =  add_for_each_block('same',Cv_Bks)
    Cv_Sd =  add_for_each_block((1, 1),Cv_Bks)
    Cv_Av =  add_for_each_block('relu',Cv_Bks)

    Po_PS =  add_for_each_block((2, 2),Cv_Bks)
    Po_Pd =  add_for_each_block('same',Cv_Bks)
    Po_Sd =  add_for_each_block((2, 2),Cv_Bks) 
    
    Dl_Num = 1
    Dl_Unt = [128]
    Dl_Reg = add_for_each_block(None,Dl_Num)
    Dl_Av =  add_for_each_block('relu',Dl_Num) 
    
    Bch_Norm = 'none'
 
    Dpout = 'none'
    Dp_Rate = 0.5
    
    Opt_Algo = 'adam'
    
    Input_Shape = (224, 224, 3)
    Input_Shape_WO_Channel = (224, 224)
    Epochs = 10
    Early_Stopping = 5
    
    Loss = 'binary_crossentropy'
    Metrics = ['accuracy', 'Precision', 'Recall']
    
    assert(Cv_Bks == len(Cv_CvLy))
    assert(Cv_Bks == len(Cv_PoLy))
    assert(Dl_Num == len(Dl_Unt))
  
    hyperparams = {'LR':LR, 'Mtm':Mtm, 'BS':BS,
                    'Cv_Bks':Cv_Bks, 'Cv_CvLy':Cv_CvLy, 'Cv_PoLy':Cv_PoLy, 
                    'Cv_KS':Cv_KS, 'Cv_Reg':Cv_Reg, 'Cv_Fil':Cv_Fil, 
                    'Cv_Pd':Cv_Pd, 'Cv_Sd':Cv_Sd, 'Cv_Av':Cv_Av, 
                    'Po_PS':Po_PS, 'Po_Pd':Po_Pd, 'Po_Sd':Po_Sd,
                    'Dl_Num':Dl_Num,'Dl_Unt':Dl_Unt, 'Dl_Reg':Dl_Reg, 'Dl_Av':Dl_Av, 
                    'Bch_Norm':Bch_Norm, 'Dpout':Dpout, 'Dp_Rate':Dp_Rate,
                    'Opt_Algo':Opt_Algo, 'Input_Shape':Input_Shape, 
                    'Input_Shape_WO_Channel':Input_Shape_WO_Channel,
                    'Epochs':Epochs, 'Early_Stopping':Early_Stopping,
                    'Loss':Loss, 'Metrics':Metrics}
    
    return hyperparams

In [11]:
def generate_model(hyperparams):
    
    Cv_Bks  = hyperparams['Cv_Bks']
    Cv_CvLy = hyperparams['Cv_CvLy']
    Cv_PoLy = hyperparams['Cv_PoLy']
    
    Cv_KS = hyperparams['Cv_KS']
    Cv_Reg = hyperparams['Cv_Reg']
    Cv_Fil = hyperparams['Cv_Fil']
    Cv_Pd = hyperparams['Cv_Pd']
    Cv_Sd = hyperparams['Cv_Sd']
    Cv_Av = hyperparams['Cv_Av']
    
    Po_PS = hyperparams['Po_PS']
    Po_Pd = hyperparams['Po_Pd']
    Po_Sd = hyperparams['Po_Sd']
    
    Dl_Num = hyperparams['Dl_Num']
    Dl_Unt = hyperparams['Dl_Unt']
    Dl_Reg = hyperparams['Dl_Reg']
    Dl_Av = hyperparams['Dl_Av']
    
    Bch_Norm = hyperparams['Bch_Norm']
    
    Dpout = hyperparams['Dpout']
    Dp_Rate = hyperparams['Dp_Rate']
    
    Input_Shape = hyperparams['Input_Shape']
    Input_Shape_WO_Channel = hyperparams['Input_Shape_WO_Channel']
    
    Epochs = hyperparams['Epochs']
    Early_Stopping = hyperparams['Early_Stopping']
    
    Loss = hyperparams['Loss']
    Metrics = hyperparams['Metrics']
                            
    Model = tf.keras.Sequential()
    
    for i in range(Cv_Bks):

        Cv_Bk_Name = 'Blk' + str(i+1)

        for j in range(Cv_CvLy[i]):
            if (i==0 and j==0):
                
                Ly_Name = Cv_Bk_Name + '_Conv' + str(j+1)
                Ly = Conv2D_Input(Cv_Fil[i], Cv_KS[i],
                                  Cv_Sd[i], Cv_Pd[i],
                                  Cv_Av[i], Cv_Reg[i],                         
                                  Ly_Name, Input_Shape)
                Model.add(Ly)

                if Bch_Norm == 'conv' or Bch_Norm == 'all':
                    Ly = Batch_Normalize()
                    Model.add(Ly)

                if Dpout == 'conv' or Dpout == 'all':
                    Ly = DropOut(Dp_Rate)
                    Model.add(Ly)

            else:
     

                Ly_Name = Cv_Bk_Name + '_Conv' + str(j+1)
                Ly = Conv2D(Cv_Fil[i], Cv_KS[i],
                            Cv_Sd[i], Cv_Pd[i],
                            Cv_Av[i], Cv_Reg[i],                         
                            Ly_Name)
                Model.add(Ly)
            
                if Bch_Norm == 'conv' or Bch_Norm == 'all':
                    Ly = Batch_Normalize()
                    Model.add(Ly)

                if Dpout == 'conv' or Dpout == 'all':
                    Ly = DropOut(Dp_Rate)
                    Model.add(Ly)

        for k in range(Cv_PoLy[i]):

            Ly_Name = Cv_Bk_Name + '_Pool' + str(k+1)
            Ly = MaxPool(Po_PS[i], Po_Sd[i],
                         Po_Pd[i], Ly_Name)
            Model.add(Ly)
        
    Model.add(Flatten('Flatten'))

    for l in range(Dl_Num):
        
        Ly_Name = 'Dense' + str(l+1)

        Ly = Dense(Dl_Unt[l], Dl_Av[l], Dl_Reg[l], Ly_Name)
        Model.add(Ly)

        if Bch_Norm == 'dense' or Bch_Norm == 'all':
            Ly = Batch_Normalize()
            Model.add(Ly)

        if Dpout == 'dense' or Dpout == 'all':
            Ly = DropOut(Dp_Rate)
            Model.add(Ly)

    Model.add(Dense(1, 'sigmoid', None, 'Output'))
    
    return Model

In [12]:
base_dir = '/home/ubuntu/kaggle/data/xray_images/Data/'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

In [13]:
hyperparams = set_hyperparameters()

In [14]:
Model = generate_model(hyperparams)

In [15]:
train_gen, test_gen = prepare_train_test_data(train_dir, test_dir, 
                                              hyperparams['BS'], hyperparams['Input_Shape_WO_Channel'])
    
compile_and_run_model(Model, hyperparams['Opt_Algo'], hyperparams['LR'], 
                      hyperparams['Mtm'], hyperparams['Loss'], 
                          hyperparams['Metrics'])
    
callbacks = prepare_callbacks(hyperparams['Early_Stopping'])

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [16]:
Model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Blk1_Conv1 (Conv2D)          (None, 224, 224, 32)      896       
_________________________________________________________________
Blk1_Conv2 (Conv2D)          (None, 224, 224, 32)      9248      
_________________________________________________________________
Blk1_Pool1 (MaxPooling2D)    (None, 112, 112, 32)      0         
_________________________________________________________________
Blk2_Conv1 (Conv2D)          (None, 112, 112, 64)      18496     
_________________________________________________________________
Blk2_Conv2 (Conv2D)          (None, 112, 112, 64)      36928     
_________________________________________________________________
Blk2_Pool1 (MaxPooling2D)    (None, 56, 56, 64)        0         
_________________________________________________________________
Blk3_Conv1 (Conv2D)          (None, 56, 56, 96)        5

In [ ]:
history = Model.fit(train_gen, validation_data= test_gen,
                    epochs = hyperparams['Epochs'], verbose=1, callbacks = callbacks)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 82 steps, validate for 10 steps
Epoch 1/10
82/82 [==============================] - 62s 751ms/step - loss: 0.3078 - accuracy: 0.8654 - Precision: 0.8699 - Recall: 0.9628 - val_loss: 0.6729 - val_accuracy: 0.7596 - val_Precision: 0.7239 - val_Recall: 0.9949
Epoch 2/10
82/82 [==============================] - 57s 698ms/step - loss: 0.1074 - accuracy: 0.9584 - Precision: 0.9711 - Recall: 0.9729 - val_loss: 1.0850 - val_accuracy: 0.7099 - val_Precision: 0.6837 - val_Recall: 0.9974
Epoch 3/10
18/82 [=====>........................] - ETA: 42s - loss: 0.0666 - accuracy: 0.9731 - Precision: 0.9813 - Recall: 0.9824

In [ ]:
Model, history, hyperparams = run_model(param_values, train_dir, test_dir, 15, 8)

In [ ]:
layer_outputs = [layer.output for layer in Model.layers[0:14]]

In [ ]:
Model.

In [ ]:
image_dir = '/home/ubuntu/kaggle/flowers/xray_images/Data/train/PNEUMONIA/'
image_path = image_dir + 'person997_virus_1678.jpeg'


# Loading the image and converting it to a numpy array for feeding it to the model. Its important to use expand_dims since our original model takes batches of images
# as input, and here we are feeding a single image to it, so the number of dimensions should match for model input.
img = image.load_img(image_path, target_size=(224, 224))
img_arr = image.img_to_array(img)
img_arr = np.expand_dims(img_arr, axis=0)
img_arr /= 255.

In [ ]:
activation_model = tf.keras.Model(inputs = Model.inputs, outputs = [layer.output for layer in Model.layers])

In [ ]:
activations = activation_model.predict(img_arr)
input_img = img_arr.reshape(224,224,3)

In [ ]:
fig = plt.figure(figsize=(22, 3))

for i in range(8):
    ax = fig.add_subplot(1, 8, i + 1)
    ax = plt.imshow(activations[0][0, :, :, i], cmap='inferno')
    plt.xticks([])
    plt.yticks([])
    fig.subplots_adjust(wspace=0.05, hspace=0.05)

In [ ]:
ax = plt.imshow(activations[6][0, :, :,11], cmap='inferno')

In [ ]:
test_normal_dir = '/home/ubuntu/kaggle/flowers/xray_images/Data/test/PNEUMONIA/'

In [ ]:
test_normal_fnames = os.listdir(test_normal_dir)

In [ ]:
length = len(test_normal_fnames)

In [ ]:
print(length)

In [ ]:
def get_image_array(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_arr = image.img_to_array(img)
    img_arr = np.expand_dims(img_arr, axis=0)
    img_arr /= 255.
    return img_arr

In [ ]:
image_path = test_normal_dir + test_normal_fnames[0]
model_input = get_image_array(image_path)
model_output = Model.predict(model_input)
model_output_all = model_output

length = len(test_normal_fnames)

for i in range(1,length):
    image_path = test_normal_dir + test_normal_fnames[i]
    model_input = get_image_array(image_path)
    model_output = Model.predict(model_input)    
    model_output_all = np.concatenate((model_output_all,model_output), axis = 0)
    
y_hat = model_output_all.reshape(length,1)
image_path = np.array(test_normal_fnames).reshape(length,1)
model_eval = np.concatenate((image_path,y_hat), axis=1)
model_eval = pd.DataFrame(model_eval, columns=['Image_Path', 'Y_hat'])
model_eval['Image_Path'] = test_normal_dir + model_eval['Image_Path']

In [ ]:
model_eval['Y_hat'] = model_eval['Y_hat'].astype(float)

In [ ]:
model_eval.hist(bins = 100)

In [ ]:
model_eval.shape

In [ ]:
viz_images = model_eval[model_eval['Y_hat']>0.8]

In [ ]:
img_path = viz_images.iloc[4]['Image_Path']

In [ ]:
img_arr = get_image_array(img_path)

In [ ]:
activation_model = tf.keras.Model(inputs = Model.inputs, outputs = [layer.output for layer in Model.layers])

In [ ]:
activations = activation_model.predict(img_arr)
input_img = img_arr.reshape(224,224,3)

In [ ]:
fig = plt.figure(figsize=(22, 3))

for i in range(8):
    ax = fig.add_subplot(1, 8, i + 1)
    ax = plt.imshow(activations[4][0, :, :, i], cmap='inferno')
    plt.xticks([])
    plt.yticks([])
    fig.subplots_adjust(wspace=0.05, hspace=0.05)

In [ ]:
plt.imshow(img_arr.reshape(224,224,3))

In [ ]:
def deprocess_image(x):
    
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1
    x += 0.5
    x = np.clip(x, 0, 1)
    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')

    return x

In [ ]:
Model.get_layer('Blk1_Conv1').output

In [ ]:
def generate_pattern(layer_name, filter_index, size=150):
    
    layer_output = Model.get_layer(layer_name).output
    
    loss = K.mean(layer_output[:, :, :, filter_index])
    
    grads = K.gradients(loss, complete_model.input)[0]
    
    grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)
    
    iterate = K.function([complete_model.input], [loss, grads])
    
    input_img_data = np.random.random((1, size, size, 3)) * 20 + 128.
    
    step = 1.
    
    for i in range(80):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step
        
    
    img = input_img_data[0]
    
    return deprocess_image(img)

In [ ]:
fig = plt.figure(figsize=(15, 12))

for img in range(30):
    ax = fig.add_subplot(5, 6, img+1)
    ax = plt.imshow(generate_pattern('conv2d_1', img))
    plt.xticks([])
    plt.yticks([])
    fig.subplots_adjust(wspace=0.05, hspace=0.05)